# MOPITT Carbon Monoxide (CO) Mixing Ratio

## Summary

## Note
By: Cheyenne Land

Tested using Python 3.8.10

## 1. Setup

In [182]:
import numpy as np
import h5py
from matplotlib import pyplot as PLT
import cartopy.crs as ccrs
import PIL
import earthaccess
import os

home_dir = os.environ['HOME']
MOPITT_dir = os.path.abspath(f"{home_dir}/MOPITT")
if not os.path.exists(MOPITT_dir):
    os.makedirs(MOPITT_dir)

%reload_ext autoreload
%autoreload 2

## 2. Search for data using earthaccess

In [ ]:
earthaccess.login()
short_name = 'MOP03JM'
version = '9'
collection = short_name + '_' + version
results = earthaccess.search_data(
    short_name= short_name,
    version = version,
    temporal=("2024-09", "2024-10")
)
print(f'{len(results[:])} file(s) found.')
print(results)

## 3. Download data

In [ ]:
# IMPORTANT NOTE:
# There are intermittent issues when donwloading from the ECS datapool. For now, this code will change from
# using the ECS datapool links to Direct Data Download links.
# This modification is temporary and will be removed once collection is in the cloud.
for i in range(len(results[:])):
    ecs_link = results[:][i]['umm']['RelatedUrls'][0]['URL']
    ddd_link = ecs_link.replace('https://l5ftl01.larc.nasa.gov/ops/misrl2l3', 'https://asdc.larc.nasa.gov/data')
    results[:][i]['umm']['RelatedUrls'][0]['URL'] = ddd_link


for r in results[:]:
    file_name = r['umm']['DataGranule']['Identifiers'][0]['Identifier']
    file_name_wo_ext = r['umm']['DataGranule']['Identifiers'][0]['Identifier'].split('.',1)[0]
    try:
        os.makedirs(f'{MOPITT_dir}/{collection}/{file_name_wo_ext}')
    except FileExistsError: 
        continue
    finally:
        if os.path.exists(f'{MOPITT_dir}/{collection}/{file_name_wo_ext}/{file_name}') == True:
            print(f'The file, {file_name} already exists. \nLocated here: ~/MOPITT/{collection}/{file_name}')
        else:
            earthaccess.download(r, f'{MOPITT_dir}/{collection}/{file_name_wo_ext}')
            print(f'File, {file_name}, downloaded here: ~/MOPITT/{collection}/{file_name_wo_ext}')

## 4. Generate plots

In [ ]:

import matplotlib.animation as animation
data_files = []
for file_name_wo_ext in os.listdir(f'{MOPITT_dir}/{collection}/'):
    for file in os.listdir(f'{MOPITT_dir}/{collection}/{file_name_wo_ext}'):
        if file.endswith(".he5"):
            data_files.append(os.path.join(f'{MOPITT_dir}/{collection}/{file_name_wo_ext}', file))

for data_file in data_files:
    with h5py.File(data_file, mode='r') as f:
        name = '/HDFEOS/GRIDS/MOP03/Data Fields/RetrievedCOMixingRatioProfileDay'

        # Slice data to get the pressure level of your choice
        #   [Longitude(Xdim):360 , Latitude(Ydim):180, Presure level:9]
        #   Pressure Level: 0 = 900 hPa, 1 = 800 hPa, 2 = 700 hPa, 3 = 600 hPa
        #   4 = 500 hPa, 5 = 400 hpa, 6 = 300 hPa, 7 = 200 hPa, 8 = 100 hPa
        d = f[name][:]
        data = np.transpose(d)

        # Retrieve the lat and lon data as well as the area of your choice
        lon = f['/HDFEOS/GRIDS/MOP03/Data Fields/Longitude'][:]
        lat = f['/HDFEOS/GRIDS/MOP03/Data Fields/Latitude'][:]

        # Turn the -9999.0 into a NaN
        masked_data = np.ma.masked_where(data <= 0, data)
        CO_mixing_ratio = data.copy()
        CO_mixing_ratio[masked_data <= 0] = np.nan

        pressure_levels = np.arange(0,9)

        file = data_file.split('/')[-1]
        file_name_wo_ext = file.split('.',1)[0]

        #Plot all graphs
        for i in pressure_levels:
            PLT.figure(figsize=(7.20,3.60))
            ax = PLT.axes(projection=ccrs.PlateCarree())
            im = PLT.contourf(lon, lat, CO_mixing_ratio[i,:,:], levels=np.linspace(0, 430, 25),
                            cmap=PLT.get_cmap('jet'), vmin=0, vmax=430,
                            transform=ccrs.PlateCarree())
            ax.coastlines()

            cb = PLT.colorbar(im, shrink = 0.76)
            cb.set_label('ppbv', fontsize=8)
            pressure = -i + 9


            PLT.title(f'{file}\n RetrievedCOMixingRatioProfileDay_{str(pressure)}00hPa',
                    fontsize=8)
            
            # save the figure in your current directory
            try:
                if os.path.exists(f'{MOPITT_dir}/{collection}/{file_name_wo_ext}/{file}_RetrievedCOMixingRatioProfileDay_{str(pressure)}00hPa.jpg') == True:
                    print(f'This plot already exits: {MOPITT_dir}/{collection}/{file_name_wo_ext}/{file}_RetrievedCOMixingRatioProfileDay_{str(pressure)}00hPa.jpg')
                else:
                    PLT.savefig(f'{MOPITT_dir}/{collection}/{file_name_wo_ext}/{file}_RetrievedCOMixingRatioProfileDay_{str(pressure)}00hPa.jpg',
                                dpi=400)
                    print(f'Plot saved here: ~/MOPITT/{collection}/{file_name_wo_ext}/{file}_RetrievedCOMixingRatioProfileDay_{str(pressure)}00hPa.jpg')
            except FileExistsError:
                continue
            PLT.close()
            PLT.clf()

## 5. Create gif

In [ ]:
frames = []
pressure = np.arange(1,10)[::-1]
for data_file in data_files:
    file = data_file.split('/')[-1]
    file_name_wo_ext = file.split('.',1)[0]
    for j in pressure:
        new_frame = PIL.Image.open(rf'{MOPITT_dir}/{collection}/{file_name_wo_ext}/{file}_RetrievedCOMixingRatioProfileDay_{str(j)}00hPa.jpg')
        frames.append(new_frame)

    try:
        if os.path.exists(f'{MOPITT_dir}/{collection}/{file_name_wo_ext}/{file_name_wo_ext}_RetrievedCOMixingRatioProfileDay_PressureLevels.gif') == True:
            print(f'This gif already exits: {MOPITT_dir}/{collection}/{file_name_wo_ext}/{file_name_wo_ext}_RetrievedCOMixingRatioProfileDay_PressureLevels.gif')
        else:
            frames[0].save(f'{MOPITT_dir}/{collection}/{file_name_wo_ext}/{file_name_wo_ext}_RetrievedCOMixingRatioProfileDay_PressureLevels.gif', format = 'GIF',
                        append_images = frames[:], save_all = True, duration = 250, loop = 0)
            print(f'GIF saved here: ~/MOPITT/{collection}/{file_name_wo_ext}/{file_name_wo_ext}_RetrievedCOMixingRatioProfileDay_PressureLevels.gif')
    except FileExistsError:
        continue

    print('Removing data file...')
    os.remove(data_file)        